# Evolving Clojure through macros

### Textual substitution


In [1]:
; Changing the value of 'a-ref' with the standard 'ref-set- function

(def a-ref (ref 0))

(dosync 
    (ref-set a-ref 1))

1

In [2]:
; Changing the value of a ref with a new macro 'sync-set'

(defmacro sync-set [r v]
    (list 'dosync 
        (list 'ref-set r v)))

(sync-set a-ref 2)

2

### The 'unless' example

In [3]:
; The general form of an if condition is as follows

"""
(if test then else)
"""

""

In [4]:
; Cheching if a number is odd with the 'odd?' built-in function

(defn exhibits-oddity?-standard [x]
    (if (odd? x)
        (println "Very odd!")))


#'user/exhibits-oddity?-standard

The testing executes as expected

In [5]:
(exhibits-oddity?-standard 11)

Very odd!


nil

In [6]:
(exhibits-oddity?-standard 10)

nil

In [17]:
; Cheching if a number is odd with an user-defined function 'unless-function'

(defn unless-function [test then]
    (if (not test)
        then))

(defn exhibits-oddity?-function [x]
    (unless-function (even? x)
        (println "Very odd, indeed!")))


#'user/exhibits-oddity?-function

THe testing shows unexpected results

In [18]:
(exhibits-oddity?-function 11)

Very odd, indeed!


nil

In [19]:
; This happens since all arguments of 'exhibits-oddity?-function' 
; are evaluated before the 'if' expression in 'unless' even begins

(exhibits-oddity?-function 10)

Very odd, indeed!


nil

In [20]:
; Cheching if a number is odd with an user-defined function 'unless-function2'
; and using an anonymous function as its 2nd argument

(defn unless-function2 [test then-thunk]
    (if (not test)
        (then-thunk)))


(defn exhibits-oddity?-function2 [x]
    (unless-function2 (even? x)
        #(println "Rather odd!")))

#'user/exhibits-oddity?-function2

In [21]:
(exhibits-oddity?-function2 11)

Rather odd!


nil

In [23]:
; This works, but wrapping the argument with an anonymous function
; is not the best solution in every case

(exhibits-oddity?-function2 10)

nil

In [25]:
; Cheching if a number is odd with an user-defined macro 'unless-macro'

(defmacro unless-macro [test then]
    (list 'if (list 'not test)
        then))

(defn exhibits-oddity?-macro [x]
    (unless-macro (even? x)
        (println "Very odd, indeed!")))


#'user/exhibits-oddity?-macro

In [26]:
(exhibits-oddity?-macro 11)

Very odd, indeed!


nil

In [27]:
(exhibits-oddity?-macro 10)

nil

In [29]:
; Checking the resulting s-expression with 'macroexpand'

(macroexpand 
    '(unless (even? x) (println "Very odd, indeed!"))) 

(if (not (even? x)) (println "Very odd, indeed!"))

### Macro templates

In [1]:
; The original unless macro

(defmacro unless [test then]
    (list 'if (list 'not test)
        then))


#'user/unless

In [2]:
; Rewriting the last macro using '`' (syntax quote character) and '~' (syntax unquote character)

(defmacro unless [test then]
    `(if (not ~test)
        ~then))


#'user/unless

In [3]:
; Generalizing the macro to take an arbitrary expression as arg

(defmacro unless [test & exprs]
    `(if (not ~test)
        (do ~exprs)))


#'user/unless

In [4]:
; Using the redefined macro in the following function
; (it works, but also throws the beloved NullPointerException)

(defn exhibits-oddity? [x]
    (unless (even? x)
        (do
            (println "Odd!")
            (println "Very odd!"))))

(exhibits-oddity? 11)

Odd!
Very odd!


Execution error (NullPointerException) at user/exhibits-oddity? (REPL:5).
null


class java.lang.NullPointerException: 

In [ ]:
; Expanding the macro to see any errors

(macroexpand-1 '(unless (even? x) 
    (println "Odd!") 
    (println "Very odd!")))



(if (clojure.core/not (even? x)) (do ((println "Odd!") (println "Very odd!"))))

```
From the output, there's an extra pair of parentheses:
    do ((println "Odd!") (println "Very odd!"))
When the expected expression is:
    do (println "Odd!") (println "Very odd!")
The first expression evaluates to:
    (nil nil)
Which is interpreted as a function call, and since the argument is nil, 
the NullPointerException is thrown.
```

In [5]:
; Usint the splice reader macro ('~@') to 'unpack' a list of args
(defmacro unless [test & exprs]
    `(if (not ~test)
        (do ~@exprs)))


#'user/unless

In [7]:
; Using the redefined macro in the following function
; (it works, and the NullPointerException is removed)

(defn exhibits-oddity? [x]
    (unless (even? x)
        (do
            (println "Odd!")
            (println "Very odd!"))))

(exhibits-oddity? 11)

Odd!
Very odd!


nil

### Generating names

In [12]:
; A badly defined macro

(defmacro def-logged-fn [fn-name args & body]
    `(defn ~fn-name ~args
        (let [now (System/currentTimeMillis)]
            (println "[" now "] Call to" (str (var ~fn-name)))
            ~@body)))


#'user/def-logged-fn

In [15]:
; Calling the last macro throws a syntax error

(def-logged-fn printname [name]
    (println "hi" name))


Syntax error macroexpanding clojure.core/let at (REPL:3:1).
user/now - failed: simple-symbol? at: [:bindings :form :local-symbol] spec: :clojure.core.specs.alpha/local-name
user/now - failed: vector? at: [:bindings :form :seq-destructure] spec: :clojure.core.specs.alpha/seq-binding-form
user/now - failed: map? at: [:bindings :form :map-destructure] spec: :clojure.core.specs.alpha/map-bindings
user/now - failed: map? at: [:bindings :form :map-destructure] spec: :clojure.core.specs.alpha/map-special-binding


class clojure.lang.Compiler$CompilerException: 

In [16]:
; When using macroexpand-1, it can be seen that 'now' 
; is being redefined as 'user/now', which isn't defined

(macroexpand-1 '(def-logged-fn printname [name]
    (println "hi" name)))


(clojure.core/defn printname [name] (clojure.core/let [user/now (java.lang.System/currentTimeMillis)] (clojure.core/println "[" user/now "] Call to" (clojure.core/str (var printname))) (println "hi" name)))

In [18]:
; Rewriting the macro to use the '#' reader macro that generates
; unique names that won’t conflict with others

(defmacro def-logged-fn [fn-name args & body]
    `(defn ~fn-name ~args
        (let [now# (System/currentTimeMillis)]
            (println "[" now# "] Call to" (str (var ~fn-name)))
            ~@body)))


#'user/def-logged-fn

In [21]:
; Calling the last macro now works as expected

(def-logged-fn printname [name]
    (println "hi" name))

(printname "deepthi")

[ 1642414687208 ] Call to #'user/printname
hi deepthi


nil